<a href="https://colab.research.google.com/github/its-rajesh/Music-Speech-Separation/blob/main/CreatingDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pydub import AudioSegment

In [ ]:
for i in range(1, 61):
    music = AudioSegment.from_file("/home/rajesh/MusicSpeechSeparartion/Dataset/music/"+str(i)+".wav", format="wav")
    speech = AudioSegment.from_file("/home/rajesh/MusicSpeechSeparartion/Dataset/speech/"+str(i)+".wav", format="wav")

    merge = music+speech
    mixture = merge.export("/home/rajesh/MusicSpeechSeparartion/Dataset/Mixture/"+str(i)+".wav", format="wav")
    
    speech = speech + 6
    overlay = speech.overlay(music, position=0)
    output = overlay.export("/home/rajesh/MusicSpeechSeparartion/Dataset/Overlay/"+str(i)+".wav", format="wav")


# Audios to STFT Arrays (.csv) files

In [2]:
import os
import numpy as np
import pandas as pd
import soundfile as sf
import librosa as lb

In [3]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [4]:
path = "/drive/My Drive/Projects/Music-Speech-Separation/Dataset"

folders = os.listdir(path)
folders

['Mixture', 'Overlay', 'speech', 'music']

In [5]:
audiofiles, samplerates = [], []
for folder in folders:
    files = os.listdir(path+'/'+folder)
    files = sorted(files)
    audio, sr = [], []
    for file in files:
        data, samplerate = sf.read(path+'/'+folder+'/'+file)
        audio.append(data)
        sr.append(samplerate)
    audiofiles.append(audio)
    samplerates.append(sr)

In [7]:
overlay = np.array(audiofiles[0])
mixture = np.array(audiofiles[1])
speech = np.array(audiofiles[2])
music = np.array(audiofiles[3])

dataset = [overlay, mixture, speech, music]

In [8]:
datasetstft = []
for data in dataset:
    spect = []
    for audio in data:
        stft = np.abs(lb.stft(audio))
        spect.append(stft)
    datasetstft.append(spect)

In [10]:
overlay_stft = np.array(datasetstft[0])
mixture_stft = np.array(datasetstft[1])
speech_stft = np.array(datasetstft[2])
music_stft = np.array(datasetstft[3])

dataset_stft = [overlay_stft, mixture_stft, speech_stft, music_stft]

In [11]:
overlay_stft.shape, mixture_stft.shape, speech_stft.shape, music_stft.shape

((50, 1025, 1292), (50, 1025, 646), (50, 1025, 646), (50, 1025, 646))

In [12]:
groundtruth_stft = []
for i in range(50):
    groundtruth_stft.append(np.hstack((speech_stft[i], music_stft[i])))

    
groundtruth_stft = np.array(groundtruth_stft)
groundtruth_stft.shape

(50, 1025, 1292)

### Creating DataFrames and CSV Files

In [19]:
csv_path = '/drive/My Drive/Projects/Music-Speech-Separation/CSVFilesTraining'

In [21]:
i = 1
for file in groundtruth_stft:
  f = pd.DataFrame(file)
  f.to_csv(csv_path+'/GroundTruth/'+str(i)+'.csv')
  i += 1

In [22]:
i = 1
for file in overlay_stft:
  f = pd.DataFrame(file)
  f.to_csv(csv_path+'/Mixture/'+str(i)+'.csv')
  i += 1